# Implementing Nerual Networks Model to classify the given image into 1 of the 10 categories of the fashion MNIST dataset

## Step 1: Importing the dataset

In [1]:
path = r"C:\NumberMNIST"

print("Path to dataset files:", path)

Path to dataset files: C:\NumberMNIST


## Step 2: Importing necessary libraries and loading the dataset

In [2]:
import numpy as np
import struct
import os

def load_images(file_path):
    with open(file_path, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows * cols)
    return images

def load_labels(file_path):
    with open(file_path, 'rb') as f:
        magic, num_labels = struct.unpack('>II', f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

os.listdir(path) # List files in the dataset directory to verify paths


['t10k-images-idx3-ubyte',
 't10k-images.idx3-ubyte',
 't10k-labels-idx1-ubyte',
 't10k-labels.idx1-ubyte',
 'train-images-idx3-ubyte',
 'train-images.idx3-ubyte',
 'train-labels-idx1-ubyte',
 'train-labels.idx1-ubyte']

In [3]:
X_train = load_images(path + r"\train-images.idx3-ubyte")
y_train = load_labels(path + r"\train-labels.idx1-ubyte")

X_test = load_images(path + r"\t10k-images.idx3-ubyte")
y_test = load_labels(path + r"\t10k-labels.idx1-ubyte")

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)
print("Train labels shape:", y_train.shape)
print("Test labels shape:", y_test.shape)

Train shape: (60000, 784)
Test shape: (10000, 784)
Train labels shape: (60000,)
Test labels shape: (10000,)


## Step 3: Data Preprocessing

**Flattening** is the process of converting a multi-dimensional input (such as a 2D image matrix) into a one-dimensional vector so that it can be fed into a fully connected neural network.

Here, flattening is the process of converting each 28×28 image into a 784-dimensional vector so that it can be used as input to a fully connected neural network.

**In this case, flattening images is not necessary as they are already in the shape (num_samples, 784)!**

In [4]:
np.min(X_train), np.max(X_train) # Check pixel value range

(0, 255)

In [5]:
# Normalize pixel values to [0, 1]
X_train= X_train / 255.0
X_test = X_test / 255.0

# Why 255.0? Because pixel values in the dataset are in the range [0, 255], 
# and dividing by 255.0 scales them to the range [0, 1], 
# which is beneficial for training neural networks.

In [6]:
print(f"Unique labels in y_train: {np.unique(y_train)}")

Unique labels in y_train: [0 1 2 3 4 5 6 7 8 9]


Neural network needs 0s and 1s, not other numbers, hence we one-hot encode the original labels

In [7]:
# One-hot encode labels

def one_hot_encode(y, num_classes):
    one_hot_matrix= np.zeros((y.shape[0], num_classes))
    for i in range(y.shape[0]):
        one_hot_matrix[i, y[i]] = 1
    return one_hot_matrix

y_train_encoded = one_hot_encode(y_train, 10)
y_test_encoded = one_hot_encode(y_test, 10)

print("One-hot encoded train labels shape:", y_train_encoded.shape)
print("One-hot encoded test labels shape:", y_test_encoded.shape)

One-hot encoded train labels shape: (60000, 10)
One-hot encoded test labels shape: (10000, 10)


**What One-Hot Encoding Does:**

If:

y = [2, 0, 3]
num_classes = 4


Then output becomes:

[[0 0 1 0]
[1 0 0 0]
[0 0 0 1]]

where index of each one in each 1-D array denotes the class (for eg., index 2 -> Class 2).

**-> One-hot encoding is required for computing cross-entropy loss in multi-class classification problems.**

Now initially, the shape of the labels were:

Train labels shape: (60000,)
Test labels shape: (10000,)

i.e., they were 1D array initally.

After one-hot encoding:

One-hot encoded train labels shape: (60000, 10)
One-hot encoded test labels shape: (10000, 10)


i.e., they are 2D matrices now.

Now the question is: **Why Do We Need 2D Matrices?**

It's because we are training on many samples at once, not one image at a time.

Neural networks use matrix multiplication to process batches efficiently.

In [8]:
# Quick dataset analysis

unique, counts = np.unique(y_train, return_counts=True)

for u, c in zip(unique, counts):
    print(f"Class {u}: {c} samples")

# zip() is a built-in Python function that lets you iterate over 
# multiple sequences (or lists) at the same time.

Class 0: 5923 samples
Class 1: 6742 samples
Class 2: 5958 samples
Class 3: 6131 samples
Class 4: 5842 samples
Class 5: 5421 samples
Class 6: 5918 samples
Class 7: 6265 samples
Class 8: 5851 samples
Class 9: 5949 samples


Here, each class has around 6000 samples, hence the dataset is **mostly balanced**.

Before we move on to the next step, we have three options for training: 

1. Batch Gradient Descent (use all 60,000 samples at once)
2. Mini-Batch Gradient Descent (use small batches like 32/64/128)
3. Stochastic Gradient Descent (1 sample at a time)

We will pick the second one here because it has faster convergence, more stable than SGD and has better generalisation.

### Step 4: Implementing Mini-Batch Training

In [1]:
# Prevents learning bias by ensuring that the model sees a 
# balanced number of samples from each class during training.

# aragne(): Creates an array of indices from 
# 0 to the number of samples in X.
# Not to be confused with range(), which creates a list of integers.

def shuffle_data(X, Y):
    indices= np.arange(X.shape[0])
    np.random.shuffle(indices) # Shuffle the indices in place

    return X[indices], Y[indices]

**NOTE:** What's reproducibility? It means that if you run the code multiple times, you will get the same results each time. This is important for debugging and comparing results.

This is achieved by np.random.seed().

In [ ]:
# Softmax function
def softmax(z):

    #keepdims=True keeps the dimensions of the output the same as the input,
    # which is important for broadcasting during division.

    # How? By subtracting the maximum value in each row of z from every element in that row,
    # we ensure that the largest value in each row becomes 0, and all other values become negative.

    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True)) # For numerical stability
    
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [8]:
# Relu activation function

def relu(z):
    return np.maximum(0, z)

In [3]:
# Forward propagation function

def forward_propagation(X, W1, b1, W2, b2):
    
    # Here, z1 represents the pre-activation output of the hidden layer,
    # which is calculated by performing a linear transformation 
    # on the input batch (X_batch)

    z1= np.dot(X, W1) + b1 # Linear transformation for hidden layer
            
    a1= relu(z1) # Calling relu() for hidden layer

    # z2 represents the pre-activation output of the output layer, 
    # which is calculated by performing a linear transformation on 
    # the activated hidden layer(a1)

    z2= np.dot(a1, W2) + b2 # Linear transformation for output layer

    # Calling softmax on z2 converts the raw output scores into probabilities
    # that sum to 1 across the output classes.

    a2= softmax(z2) # Softmax activation for output layer

    return a1, a2

In [4]:
# Defining categorical cross-entropy loss function

def cat_cross_entropy_loss(m, y_batch, a2):
    # m is the number of samples in the batch
    # y_batch is the true one-hot encoded labels for the batch
    # a2 is the predicted probabilities from the output layer

    # Adding a small epsilon to prevent log(0)
    epsilon = 1e-8

    # Calculate the loss using the categorical cross-entropy formula
    loss = -np.sum(y_batch * np.log(a2 + epsilon)) / m

    return loss

In [ ]:
# Defining backward propagation function

def back_prop(a1, a2, X_batch, y_batch, m, w2, z1):

    dZ2= a2 - y_batch # Gradient of loss wrt z2 (output layer pre-activation)
    
    dW2= np.dot(a1.T, dZ2) / m # Gradient of loss wrt to W2
    db2= np.sum(dZ2, axis=0, keepdims=True) / m # Gradient of loss wrt b2

    dA1= np.dot(dZ2, w2.T) # Gradient of loss wrt a1 (hidden layer activation)
    dZ1= dA1 * (z1 > 0)     # Gradient of loss wrt z1 (hidden layer pre-activation), applying ReLU derivative

    dW1= np.dot(X_batch.T, dZ1) / m # Gradient of loss wrt W1
    db1= np.sum(dZ1, axis= 0, keepdims= True) / m # Gradient of loss wrt b1

    return dW1, db1, dW2, db2

**NOTE:** The activated output of the hidden layer represents learned feature transformations, whereas the output layer produces class probabilities via the softmax function, which are used for final classification.

In [7]:
# Training function

def train(batch_size, epochs, lr):

    input_size= X_train.shape[1] # 784 for 28x28 images
    output_size= y_train_encoded.shape[1] # 10 for 10 classes (0 - 9)

    # He initialisation for weights (important for ReLU activation)
    np.random.seed(100) # Ensure reproducibility

    W1= np.random.randn(input_size, 100) * np.sqrt(2 / input_size) # 100 neurons in hidden layer
    b1= np.zeros((1, 100)) # Bias for hidden layer

    W2= np.random.randn(100, output_size) * np.sqrt(2 / 100) # Output layer weights
    b2= np.zeros((1, output_size)) # Bias for output layer

    history= [] # To store loss history

    for epoch in range(epochs):

        # Shuffle training data at the beginning of each epoch
        indices= np.random.permutation(X_train.shape[0]) # Shuffled indices for the entire dataset

        X_train_shuffled= X_train[indices]
        y_train_shuffled= y_train_encoded[indices]

        epoch_loss= 0

        for i in range(0, X_train.shape[0], batch_size):

            X_batch= X_train_shuffled[i: i + batch_size]
            y_batch= y_train_shuffled[i: i+ batch_size]

            # Calling forward_propagation() to get activations
            a1, a2= forward_propagation(X_batch, W1, b1, W2, b2)

            # Here, 
            
            # a1: The activated output of the hidden layer, 
            # which is used in backpropagation to compute gradients for the 
            # weights and biases of both layers.

            # a2: The predicted probabilities from the output layer, 
            # which is used to calculate gradients in backpropagation.

            # Calculate loss for the current batch
            epoch_loss += cat_cross_entropy_loss(X_batch.shape[0], y_batch, a2)

            # Now, forward propagation computes predictions and computes loss
            # but it DOES NOT UPDATE WEIGHTS! It only answers: 
            # "GIVEN current weights, what is the prediction?"

            # Hence we need back prop as well

            # Calling back_prop() to compute gradients
            dW1, db1, dW2, db2= back_prop(a1, a2, X_batch, y_batch, X_batch.shape[0], W2, np.dot(X_batch, W1) + b1)

            # Update weights and biases using gradient descent
            W1 -= lr * dW1
            b1 -= lr * db1
            W2 -= lr * dW2
            b2 -= lr * db2

        history.append(epoch_loss / (X_train.shape[0] / batch_size)) # Average loss per epoch
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {history[-1]:.4f}")

    return W1, b1, W2, b2, history

### Step 5: Predicting and Evaluating